In [54]:
import os
import keras
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.layers import Flatten, Dense
from keras.models import Model
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator


from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.regularizers import l2
from keras import backend as K





In [55]:
DATASET_PATH  = 'C:/Users/195313/Documents/GitHub/4-th-ML100Days/Day101_HW/4th-cupoy-ml-100-marathon-finalexam/image_data/train'
TEST_PATH = 'C:/Users/195313/Documents/GitHub/4-th-ML100Days/Day101_HW/4th-cupoy-ml-100-marathon-finalexam/image_data/test'
OUTPUT_PATH = 'C:/Users/195313/Documents/GitHub/4-th-ML100Days/Day101_HW'
# NUM_CLASSES = 5
# NUM_EPOCHS = 32
SEED = 77
# saved model
WEIGHTS_FINAL = 'model-InceptionResNetV2.h5'
categories=os.listdir(DATASET_PATH)

In [56]:
# train_datagen = ImageDataGenerator(
#     # set input mean to 0 over the dataset
#     featurewise_center=False,
#     # set each sample mean to 0
#     samplewise_center=False,
#     # divide inputs by std of dataset
#     featurewise_std_normalization=False,
#     # divide each input by its std
#     samplewise_std_normalization=False,
#     # apply ZCA whitening
#     zca_whitening=False,
#     # epsilon for ZCA whitening
#     zca_epsilon=1e-06,
#     # randomly rotate images in the range (deg 0 to 180)
#     rotation_range=0,
#     # randomly shift images horizontally
#     width_shift_range=0.1,
#     # randomly shift images vertically
#     height_shift_range=0.1,
#     # set range for random shear
#     shear_range=0.,
#     # set range for random zoom
#     zoom_range=0.,
#     # set range for random channel shifts
#     channel_shift_range=0.,
#     # set mode for filling points outside the input boundaries
#     fill_mode='nearest',
#     # value used for fill_mode = "constant"
#     cval=0.,
#     # randomly flip images
#     horizontal_flip=True,
#     # randomly flip images
#     vertical_flip=False,
#     # set rescaling factor (applied before any other transformation)
#     rescale=None,
#     # set function that will be applied on each input
#     preprocessing_function=None,
#     # image data format, either "channels_first" or "channels_last"
#     data_format=None,
#     # fraction of images reserved for validation (strictly between 0 and 1)
#     validation_split=0.0)

In [57]:
# Image preprocess
train_datagen = ImageDataGenerator(rotation_range=30, width_shift_range=0.125, height_shift_range=0.125, zoom_range=0.125, horizontal_flip=True,
                                   validation_split=0.2, rescale=1. / 255)
train_batches = train_datagen.flow_from_directory(DATASET_PATH, subset = 'training', seed = SEED)
valid_batches = train_datagen.flow_from_directory(DATASET_PATH, subset = 'validation', seed = SEED)
for cls, idx in train_batches.class_indices.items():
    print('Class #{} = {}'.format(idx, cls))

Found 2260 images belonging to 5 classes.
Found 563 images belonging to 5 classes.
Class #0 = daisy
Class #1 = dandelion
Class #2 = rose
Class #3 = sunflower
Class #4 = tulip


In [58]:
# 影像輸入的維度
# input_shape = train_batches.shape[1:]
input_shape=(256, 256, 3)
# 訓練用的超參數
batch_size = 128  
epochs = 32
data_augmentation = True
num_classes = 5
n = 9 # 使用 ResNet-56 的網路架構

# 使用的 ResNet 模型版本
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 1

# 計算不同 ResNet 版本對應的網路深度，此處都是根據 paper 的定義來計算
depth = n * 6 + 2

# 模型的名稱
model_type = 'ResNet%dv%d' % (depth, version)

In [59]:
# 學習率動態調整。當跑到第幾個 epcoh 時，根據設定修改學習率。這邊的數值都是參考原 paper
def lr_schedule(epoch):
    """Learning Rate Schedule
    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.
    # Arguments
        epoch (int): The number of epochs
    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [60]:
# # 使用 resnet_layer 來建立我們的 ResNet 模型
# def resnet_layer(inputs,
#                  num_filters=16,
#                  kernel_size=3,
#                  strides=1,
#                  activation='relu',
#                  batch_normalization=True,
#                  conv_first=True):
#     """2D Convolution-Batch Normalization-Activation stack builder
#     # Arguments
#         inputs (tensor): input tensor from input image or previous layer
#         num_filters (int): Conv2D number of filters
#         kernel_size (int): Conv2D square kernel dimensions
#         strides (int): Conv2D square stride dimensions
#         activation (string): activation name
#         batch_normalization (bool): whether to include batch normalization
#         conv_first (bool): conv-bn-activation (True) or
#             bn-activation-conv (False)
#     # Returns
#         x (tensor): tensor as input to the next layer
#     """
#     # 建立卷積層
#     conv = Conv2D(num_filters,
#                   kernel_size=kernel_size,
#                   strides=strides,
#                   padding='same',
#                   kernel_initializer='he_normal',
#                   kernel_regularizer=l2(1e-4))

#     # 對輸入進行卷機，根據 conv_first 來決定 conv. bn, activation 的順序
#     x = inputs
#     if conv_first:
#         x = conv(x)
#         if batch_normalization:
#             x = BatchNormalization()(x)
#         if activation is not None:
#             x = Activation(activation)(x)
#     else:
#         if batch_normalization:
#             x = BatchNormalization()(x)
#         if activation is not None:
#             x = Activation(activation)(x)
#         x = conv(x)
#     return x

In [61]:
# # Resnet v1 共有三個 stage，每經過一次 stage，影像就會變小一半，但 channels 數量增加一倍。ResNet-20 代表共有 20 層 layers，疊越深參數越多
# def resnet_v1(input_shape, depth, num_classes=10):
#     """ResNet Version 1 Model builder [a]
#     Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
#     Last ReLU is after the shortcut connection.
#     At the beginning of each stage, the feature map size is halved (downsampled)
#     by a convolutional layer with strides=2, while the number of filters is
#     doubled. Within each stage, the layers have the same number filters and the
#     same number of filters.
#     Features maps sizes:
#     stage 0: 32x32, 16
#     stage 1: 16x16, 32
#     stage 2:  8x8,  64
#     The Number of parameters is approx the same as Table 6 of [a]:
#     ResNet20 0.27M
#     ResNet32 0.46M
#     ResNet44 0.66M
#     ResNet56 0.85M
#     ResNet110 1.7M
#     # Arguments
#         input_shape (tensor): shape of input image tensor
#         depth (int): number of core convolutional layers
#         num_classes (int): number of classes (CIFAR10 has 10)
#     # Returns
#         model (Model): Keras model instance
#     """
#     if (depth - 2) % 6 != 0:
#         raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
#     # 模型的初始設置，要用多少 filters，共有幾個 residual block （組成 ResNet 的單元）
#     num_filters = 16
#     num_res_blocks = int((depth - 2) / 6)
    
#     # 建立 Input layer
#     inputs = Input(shape=input_shape)
    
#     # 先對影像做第一次卷機
#     x = resnet_layer(inputs=inputs)
    
#     # 總共建立 3 個 stage
#     for stack in range(3):
#         # 每個 stage 建立數個 residual blocks (數量視你的層數而訂，越多層越多 block)
#         for res_block in range(num_res_blocks):
#             strides = 1
#             if stack > 0 and res_block == 0:  # first layer but not first stack
#                 strides = 2  # downsample
#             y = resnet_layer(inputs=x,
#                              num_filters=num_filters,
#                              strides=strides)
#             y = resnet_layer(inputs=y,
#                              num_filters=num_filters,
#                              activation=None)
#             if stack > 0 and res_block == 0:  # first layer but not first stack
#                 # linear projection residual shortcut connection to match
#                 # changed dims
#                 x = resnet_layer(inputs=x,
#                                  num_filters=num_filters,
#                                  kernel_size=1,
#                                  strides=strides,
#                                  activation=None,
#                                  batch_normalization=False)
#             x = keras.layers.add([x, y]) # 此處把 featuremaps 與 上一層的輸入加起來 (欲更了解結構需閱讀原論文)
#             x = Activation('relu')(x)
#         num_filters *= 2

#     # 建立分類
#     # 使用 average pooling，且 size 跟 featuremaps 的 size 一樣 （相等於做 GlobalAveragePooling）
#     x = AveragePooling2D(pool_size=8)(x)
#     y = Flatten()(x)
    
#     # 接上 Dense layer 來做分類
#     outputs = Dense(num_classes,
#                     activation='softmax',
#                     name='softmax')(y)

#     # 建立模型
#     model = Model(inputs=inputs, outputs=outputs)
#     return model

In [62]:
# Build Model
net = InceptionResNetV2(include_top=False, input_shape=(256, 256, 3))
x = net.output
x = Flatten()(x)
# Add Dense layer, each probability by softmax
output_layer = Dense(NUM_CLASSES, activation='softmax', name='softmax')(x)

net_final = Model(inputs=net.input, outputs=output_layer)
net_final.summary()

net_final.compile(optimizer=Adam(lr=lr_schedule(0)), loss='categorical_crossentropy', metrics=['accuracy'])

# 建立 ResNet v1 模型
# model = resnet_v1(input_shape=input_shape, depth=depth)

# 編譯模型，使用 Adam 優化器並使用學習率動態調整的函數，０代表在第一個 epochs
# model.compile(loss='categorical_crossentropy',
#               optimizer=Adam(lr=lr_schedule(0)),
#               metrics=['accuracy'])
# model.summary()
# print(model_type)

# 使用動態調整學習率
lr_scheduler = LearningRateScheduler(lr_schedule)

# 使用自動降低學習率 (當 validation loss 連續 5 次沒有下降時，自動降低學習率)
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)
# 設定 callbacks
callbacks = [lr_reducer, lr_scheduler]


print('Using real-time data augmentation.')

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_432 (Conv2D)             (None, 127, 127, 32) 864         input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_424 (BatchN (None, 127, 127, 32) 96          conv2d_432[0][0]                 
__________________________________________________________________________________________________
activation_424 (Activation)     (None, 127, 127, 32) 0           batch_normalization_424[0][0]    
____________________________________________________________________________________________

activation_516 (Activation)     (None, 14, 14, 192)  0           batch_normalization_516[0][0]    
__________________________________________________________________________________________________
activation_519 (Activation)     (None, 14, 14, 192)  0           batch_normalization_519[0][0]    
__________________________________________________________________________________________________
block17_5_mixed (Concatenate)   (None, 14, 14, 384)  0           activation_516[0][0]             
                                                                 activation_519[0][0]             
__________________________________________________________________________________________________
block17_5_conv (Conv2D)         (None, 14, 14, 1088) 418880      block17_5_mixed[0][0]            
__________________________________________________________________________________________________
block17_5 (Lambda)              (None, 14, 14, 1088) 0           block17_4_ac[0][0]               
          

__________________________________________________________________________________________________
conv2d_602 (Conv2D)             (None, 6, 6, 256)    172032      activation_593[0][0]             
__________________________________________________________________________________________________
batch_normalization_591 (BatchN (None, 6, 6, 192)    576         conv2d_599[0][0]                 
__________________________________________________________________________________________________
batch_normalization_594 (BatchN (None, 6, 6, 256)    768         conv2d_602[0][0]                 
__________________________________________________________________________________________________
activation_591 (Activation)     (None, 6, 6, 192)    0           batch_normalization_591[0][0]    
__________________________________________________________________________________________________
activation_594 (Activation)     (None, 6, 6, 256)    0           batch_normalization_594[0][0]    
__________

In [ ]:
# Train
net_final.fit_generator(train_batches, validation_data = valid_batches, epochs = epochs, verbose=1, workers=4,
                    callbacks=callbacks)
# 訓練模型囉！
# model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
#                     steps_per_epoch=int(len(x_train)//batch_size),
#                     validation_data=(x_test, y_test),
#                     epochs=epochs, verbose=1, workers=4,
#                     callbacks=callbacks)
# Store Model
net_final.save(WEIGHTS_FINAL)
# 評估我們的模型
# scores = model.evaluate(x_test, y_test, verbose=1)
# print('Test loss:', scores[0])
# print('Test accuracy:', scores[1])

Epoch 1/32
Learning rate:  0.001
70/71 [============================>.] - ETA: 1:00 - loss: 1.4937 - accuracy: 0.6827

In [ ]:
from keras.preprocessing import image
out = np.array(['id', 'flower_class'])
testfiles=os.listdir(TEST_PATH)
for testfile in testfiles:
    filename = testfile.split('.')[0]
    img = image.load_img(TEST_PATH+'/'+testfile,target_size=(256, 256))
    if img is None:
        continue
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    x = x /255.
    pred = net_final.predict(x)[0]
    tof=np.argmax(pred)
    out = np.vstack((out,[filename, tof]))

pd.DataFrame(out).to_csv(OUTPUT_PATH+'/prediction.csv',index=False,header=False)